In [1]:
import datetime
import numpy as np 
import time
import re
from datetime import datetime
import pickle
import string
import pandas as pd
from copy import deepcopy
import operator
import matplotlib.pyplot as plt
import os

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import ast
from bs4 import BeautifulSoup
import urllib.request
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import smtplib
from email.mime.text import MIMEText

In [2]:
def scrap_coin_gecko():
    
    url = f"https://www.coingecko.com/en/coins/recently_added"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    r = requests.get(url, headers)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    soup_tag = soup.find_all("a", attrs={"class": "d-lg-none font-bold"})
    
    coin_symbol_list = []
    for element in soup_tag:
        
        list_letters = re.findall('([A-Z])', str(element))
        coin_symbol = "".join(list_letters)
        
        coin_symbol_list.append(coin_symbol)
    
    return coin_symbol_list

In [3]:
def scrap_coinmarket():
    
    url = f"https://coinmarketcap.com/new/"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    r = requests.get(url, headers)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    soup_tag = soup.find_all("img", attrs={"class": "coin-logo"})

    coin_symbol_list = []
    for element in soup_tag:
        
        list_letters = re.findall('([A-Z])', str(element))
        coin_symbol = "".join(list_letters)
        
        coin_symbol_list.append(coin_symbol)
    
    return coin_symbol_list, soup_tag

In [4]:
def scrap_watchlist(soup_tag, symbol):
    
    list_tag = soup_tag.find_all("a", attrs={"class": "cmc-link"})
    ref = [str(s["href"]) for i, s in enumerate(list_tag) if "symbol" in str(s)][0]
    
    url = f"https://coinmarketcap.com{ref}"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    r = requests.get(url, headers)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    soup_tag = soup.find_all("div", attrs={"class": "namePill"})
    watchlist_num = int(re.findall(r"[-]?\d+[.]?\d+", str(soup_tag[2]))[0])
    
    return watchlist_num
    

In [5]:
def get_latest_coin_via_coinmarket_api():
    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

    parameters = {
      'start':'1',
      'limit':'1',
      'sort':'date_added',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '#########',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        blockchain_platform = data["data"][0]["platform"]["name"]
        symbol = data["data"][0]["symbol"]
        token_address = data["data"][0]["platform"]["token_address"]
        
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
  
    return blockchain_platform, symbol, token_address 
    
    

# main (using api)

In [24]:
bool_coin_market = True

In [25]:
symbol_coin_market = "BOB"

In [ ]:
while bool_coin_market:
    
    time.sleep(60)
    
    symbol_gecko_list = scrap_coin_gecko()
    
    blockchain_platform, symbol, token_address = get_latest_coin_via_coinmarket_api()
    
    if symbol_coin_market != symbol:
        
        symbol_coin_market = symbol
        
        is_already_in = symbol_coin_market in symbol_gecko_list
    
        if (not is_already_in) and (blockchain_platform == 'Binance Smart Chain (BEP20)'):
            bool_coin_market = False
        
#     symbol, soup_tag = scrap_coinmarket()
    
#     if symbol_coin_market != symbol:

#         is_already_in = (symbol_coin_market in symbol_gecko_list)
#         symbol_coin_market = symbol
    
#         if (not is_already_in):

#             watchlist_num = scrap_watchlist(soup_tag, symbol)
#             bool_coin_market = False
            
#             print(symbol, watchlist_num)

print(blockchain_platform)
print(symbol)
print(token_address)

for _ in range(50):
    os.system("beep -f 555 -l 460")
    time.sleep(0.1)


# main (using web scraping)

In [14]:
bool_coin_market = True
bool_coin_gecko = True

In [15]:
symbol_coin_market = "DFP"
#symbol_coin_gecko = "SQUID"

In [16]:
while bool_coin_gecko and bool_coin_market:
    
    time.sleep(1)
    
    symbol_marketcap_list, _ = scrap_coinmarket()
    symbol_gecko_list = scrap_coin_gecko()
    
    symbol_marketcap = symbol_marketcap_list[0]
    symbol_gecko = symbol_gecko_list[0]

    if symbol_marketcap != symbol_coin_market:
        is_already_in = symbol_marketcap in symbol_gecko_list
        
        symbol_coin_market = symbol_marketcap
    
        if not is_already_in:
            bool_coin_market = False
            print("COIN MARKET", symbol_coin_market)
        
#     if symbol_gecko != symbol_coin_gecko:
#         is_already_in = symbol_gecko in symbol_marketcap_list
#         symbol_coin_gecko = symbol_gecko
    
#         if not is_already_in:
#             bool_coin_gecko = False
#             print("COIN GECKO", symbol_coin_gecko)
    
for _ in range(50):
    os.system("beep -f 555 -l 460")
    time.sleep(0.1)

COIN MARKET M
